In [1]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast

In [2]:
# STEPS to get data

# get arr of notes
# get arr of keywords
# split note in arr
# for each word in keyword list, get count of it in arr (word_list.count(curr_keyword))
# also store which keywords are matched in arr
# have df (with cols patient id, encounter id, note TXT, keyword count, list of flagged keywords)
# to get # of general encouters --> do len(df[df["keyword_count"] >= 1])
# to get # of such encounters per patient
    # get list of unique patient ids
    # for each one, do len(df[df["patient_id"] == curr_patient_id]["encounter_id"])
    
# REPRESENTING data

# exported dataframe to csv with cols patient_id, num encounters keywords >= 1
# exported dataframe to csv that has general encounters where keywords >= 1, cols: patient id, encounter id, note txt, keyword count, matched keywords
# Summary Statistics: number of encounters with keywords >= 1, number of average encounters per patient with such encounters

In [3]:
df1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_1.csv")
#df2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_2.csv")
# df3 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_3.csv")
# df4 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_4.csv")
# df5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_5.csv")

In [4]:
#frames = [df1, df2]
notes = df1

In [5]:
notes["NoteTXT"] = notes["NoteTXT"].str.replace('\s+', ' ', regex=True)
notes['NoteTXT'] = '-------------------------------------------------------------------------------------------------------------' + notes['NoteTXT'].astype(str)+'-------------------------------------------------------------------------------------------------------------'

In [6]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\Sentence-Analysis\EDA\Getting_Data\keywords.csv")

In [7]:
regex

,REGEX
0,dementia
1,cognition
2,memory
3,CDR
4,MMSE
5,MOCA
6,Alzheimer
7,cognitive\s*impairment
8,MCI
9,cerebellar


In [8]:
notes

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT
0,100000272,10040002098,Z6352398,3.048340e+09,1999-11-17 00:00:00.0000000,System Generated,330550893,Op Note,1,3.164291e+08,1.0,----------------------------------------------...
1,100000272,10040002098,Z6352398,3.052785e+09,2009-03-15 00:00:00.0000000,System Generated,337968894,Discharge Summary,1,3.236400e+08,1.0,----------------------------------------------...
2,100000272,10040002098,Z6352398,3.056972e+09,2001-02-19 00:00:00.0000000,System Generated,353890517,Discharge Summary,1,3.393630e+08,1.0,----------------------------------------------...
3,100000272,10040002098,Z6352398,3.076013e+09,2015-06-12 00:00:00.0000000,Office Visit,464327283,MR AVS Snapshot,1,4.466111e+08,1.0,----------------------------------------------...
4,100000272,10040002098,Z6352398,3.076013e+09,2015-06-12 00:00:00.0000000,Office Visit,464415371,Progress Notes,1,4.466893e+08,1.0,----------------------------------------------...
...,...,...,...,...,...,...,...,...,...,...,...,...
1079066,100365175,10059358563,Z8714148,3.375703e+09,2021-05-26 00:00:00.0000000,Patient Message,5878295268,Telephone Encounter,1,5.876515e+09,1.0,----------------------------------------------...
1079067,100365175,10059358563,Z8714148,3.378998e+09,2021-06-14 00:00:00.0000000,Letter (Out),5966394734,Letter,1,5.965382e+09,1.0,----------------------------------------------...
1079068,100365175,10059358563,Z8714148,3.379970e+09,2021-06-17 00:00:00.0000000,Patient Message,5994021567,Telephone Encounter,1,5.993225e+09,1.0,----------------------------------------------...
1079069,100365175,10059358563,Z8714148,3.381402e+09,2021-06-23 00:00:00.0000000,Refill,6016856527,Telephone Encounter,1,6.016268e+09,1.0,----------------------------------------------...


In [9]:
locations = []
for r in tqdm(notes['NoteTXT']):
    try:
        locations.append([match.span() \
                  for match in re.finditer(re.compile("|".join(regex["REGEX"]), 
                                                  re.IGNORECASE),r)])
    except:
        locations.append([])

notes["regex_location"] = locations
notes.to_csv("apoe33_notes_1_w_locations.csv")

100%|██████████████████████████████████████████████████████████████████████| 1079071/1079071 [14:05<00:00, 1276.68it/s]


In [10]:
notes.head(100)

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location
0,100000272,10040002098,Z6352398,3.048340e+09,1999-11-17 00:00:00.0000000,System Generated,330550893,Op Note,1,3.164291e+08,1.0,----------------------------------------------...,[]
1,100000272,10040002098,Z6352398,3.052785e+09,2009-03-15 00:00:00.0000000,System Generated,337968894,Discharge Summary,1,3.236400e+08,1.0,----------------------------------------------...,[]
2,100000272,10040002098,Z6352398,3.056972e+09,2001-02-19 00:00:00.0000000,System Generated,353890517,Discharge Summary,1,3.393630e+08,1.0,----------------------------------------------...,[]
3,100000272,10040002098,Z6352398,3.076013e+09,2015-06-12 00:00:00.0000000,Office Visit,464327283,MR AVS Snapshot,1,4.466111e+08,1.0,----------------------------------------------...,[]
4,100000272,10040002098,Z6352398,3.076013e+09,2015-06-12 00:00:00.0000000,Office Visit,464415371,Progress Notes,1,4.466893e+08,1.0,----------------------------------------------...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,100000272,10040002098,Z6352398,3.161459e+09,2017-06-23 00:00:00.0000000,Hospital Encounter,1562580628,ED Triage Notes,1,1.508448e+09,1.0,----------------------------------------------...,[]
96,100000272,10040002098,Z6352398,3.161459e+09,2017-06-23 00:00:00.0000000,Hospital Encounter,1562613299,ED Provider Notes,1,1.508556e+09,1.0,----------------------------------------------...,"[(3947, 3950)]"
97,100000272,10040002098,Z6352398,3.161459e+09,2017-06-23 00:00:00.0000000,Hospital Encounter,1562623720,ED Notes,1,1.508494e+09,1.0,----------------------------------------------...,[]
98,100000272,10040002098,Z6352398,3.161459e+09,2017-06-23 00:00:00.0000000,Hospital Encounter,1562660243,ED Provider Notes,1,1.508533e+09,1.0,----------------------------------------------...,[]


In [11]:
l = []
for i in tqdm(range(len(notes["regex_location"]))):
    s = []
    try:
        for j in notes["regex_location"][i]:
            s.append(notes["NoteTXT"][i][j[0]-100 : j[0]+100])
    except:
        pass
    l.append(s)
    
    
notes["regex_sent"] = l

100%|█████████████████████████████████████████████████████████████████████| 1079071/1079071 [00:50<00:00, 21273.74it/s]


In [14]:
notes.to_csv("apoe33_notes_1_w_locations_and_sequences.csv")

In [15]:
notes.head(100)

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent
0,100000272,10040002098,Z6352398,3.048340e+09,1999-11-17 00:00:00.0000000,System Generated,330550893,Op Note,1,3.164291e+08,1.0,----------------------------------------------...,[],[]
1,100000272,10040002098,Z6352398,3.052785e+09,2009-03-15 00:00:00.0000000,System Generated,337968894,Discharge Summary,1,3.236400e+08,1.0,----------------------------------------------...,[],[]
2,100000272,10040002098,Z6352398,3.056972e+09,2001-02-19 00:00:00.0000000,System Generated,353890517,Discharge Summary,1,3.393630e+08,1.0,----------------------------------------------...,[],[]
3,100000272,10040002098,Z6352398,3.076013e+09,2015-06-12 00:00:00.0000000,Office Visit,464327283,MR AVS Snapshot,1,4.466111e+08,1.0,----------------------------------------------...,[],[]
4,100000272,10040002098,Z6352398,3.076013e+09,2015-06-12 00:00:00.0000000,Office Visit,464415371,Progress Notes,1,4.466893e+08,1.0,----------------------------------------------...,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,100000272,10040002098,Z6352398,3.161459e+09,2017-06-23 00:00:00.0000000,Hospital Encounter,1562580628,ED Triage Notes,1,1.508448e+09,1.0,----------------------------------------------...,[],[]
96,100000272,10040002098,Z6352398,3.161459e+09,2017-06-23 00:00:00.0000000,Hospital Encounter,1562613299,ED Provider Notes,1,1.508556e+09,1.0,----------------------------------------------...,"[(3947, 3950)]",[cular: Normal rate. Pulmonary/Chest: Effort n...
97,100000272,10040002098,Z6352398,3.161459e+09,2017-06-23 00:00:00.0000000,Hospital Encounter,1562623720,ED Notes,1,1.508494e+09,1.0,----------------------------------------------...,[],[]
98,100000272,10040002098,Z6352398,3.161459e+09,2017-06-23 00:00:00.0000000,Hospital Encounter,1562660243,ED Provider Notes,1,1.508533e+09,1.0,----------------------------------------------...,[],[]


In [16]:
notes['regex_sent'] = notes['regex_sent'].astype(str)
notes['regex_sent'] = notes['regex_sent'].str.replace('[', '')
notes['regex_sent'] = notes['regex_sent'].str.replace(']', '')
notes = notes.replace(r'^\s*$', np.NaN, regex=True)

In [17]:
notes = notes[notes['regex_sent'].notna()]

In [18]:
notes['regex_sent'] = notes['regex_sent'].str.replace('"', "'")

In [19]:
notes = (notes.assign(regex_sent = notes['regex_sent'].str.split("', '"))
         .explode('regex_sent')
         .reset_index(drop=True))

In [20]:
l = []
for note in tqdm(notes["regex_sent"]):
    try:
        l.append(re.findall(re.compile("|".join(regex["REGEX"]), re.IGNORECASE), note))
    except:
        l.append([])

notes["regex_match"] = l

100%|███████████████████████████████████████████████████████████████████████| 270297/270297 [00:25<00:00, 10577.82it/s]


In [21]:
notes.to_csv("apoe33_notes_1_w_everything.csv")

In [23]:
notes

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match
0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,Office Visit,471433064,Progress Notes,1,4.532520e+08,1.0,----------------------------------------------...,"[(5850, 5858)]",'ncer. She passed secondary to this. He does n...,[cerebral]
1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,Office Visit,513142600,Progress Notes,1,4.924833e+08,1.0,----------------------------------------------...,"[(3643, 3649)]",'a. Skin: No rashes or lesions. Neurological: ...,[memory]
2,100000272,10040002098,Z6352398,3.082899e+09,2015-10-15 00:00:00.0000000,Office Visit,640846456,Progress Notes,1,6.127209e+08,1.0,----------------------------------------------...,"[(6049, 6057)]",'ncer. She passed secondary to this. He does n...,[cerebral]
3,100000272,10040002098,Z6352398,3.085967e+09,2015-08-20 00:00:00.0000000,Office Visit,563095698,Progress Notes,1,5.395701e+08,1.0,----------------------------------------------...,"[(3810, 3816)]",'a. Skin: No rashes or lesions. Neurological: ...,[memory]
4,100000272,10040002098,Z6352398,3.094749e+09,2015-10-14 00:00:00.0000000,Abstract,638983522,Progress Notes,1,6.109901e+08,1.0,----------------------------------------------...,"[(5554, 5562)]",'ncer. She passed secondary to this. He does n...,[cerebral]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270292,100365175,10059358563,Z8714148,3.360340e+09,2021-04-23 00:00:00.0000000,Telemedicine,5693313879,Progress Notes,1,5.690061e+09,1.0,----------------------------------------------...,"[(964, 970)]",'he past 3 years. Interval events He was last ...,[memory]
270293,100365175,10059358563,Z8714148,3.363845e+09,2021-05-10 00:00:00.0000000,Office Visit,5784497372,Progress Notes,1,5.781962e+09,1.0,----------------------------------------------...,"[(391, 400)]",' and a moderate mixed hearing loss rising to ...,[cognition]
270294,100365175,10059358563,Z8714148,3.363845e+09,2021-05-10 00:00:00.0000000,Office Visit,5784981138,Progress Notes,1,5.782449e+09,1.0,----------------------------------------------...,"[(1194, 1203)]",'ge of Onset • Other Mother frequent falls • O...,[Alzheimer]
270295,100365175,10059358563,Z8714148,3.366388e+09,2021-04-22 00:00:00.0000000,Follow-Up,5684835347,Progress Notes,1,5.681518e+09,1.0,----------------------------------------------...,"[(672, 678), (7858, 7867)]",'n recent months. He even thinks that on May 2...,[memory]


In [24]:
notes['regex_match'] = notes['regex_match'].astype(str)
notes['regex_match'] = notes['regex_match'].str.replace("\[\]", '')
notes = notes.replace(r'^\s*$', np.NaN, regex=True)

In [25]:
notes = notes[notes['regex_match'].notna()]

In [26]:
notes['regex_sent'] = notes['regex_sent'].str.split(' ',1).str[1].str.rpartition(' ')[0]

In [27]:
import ast
notes.regex_match = notes.regex_match.apply(ast.literal_eval)

In [28]:
notes['regex_sent'] = notes['regex_sent'].apply(json.dumps)
notes['regex_match'] = notes['regex_match'].apply(json.dumps)

In [61]:
x = notes['regex_match'][97]
x = x.split(" ")
x

['["Memory",', '"dementia",', '"MOCA"]']

In [62]:
print(len(x))

3


In [64]:
def regex_length(row):
    x = row['regex_match']
    x= x.split(" ")
    return len(x)

notes['match_count'] = notes.apply(regex_length, axis = 'columns')

In [ ]:
#notes['match_count'] = notes.apply(regex_length, axis='columns')

In [29]:
notes.to_csv("apoe_1_FINAL.csv")

In [66]:
notes.head(100)

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count
0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,Office Visit,471433064,Progress Notes,1,4.532520e+08,1.0,----------------------------------------------...,"[(5850, 5858)]","""She passed secondary to this. He does not kno...","[""cerebral""]",1
1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,Office Visit,513142600,Progress Notes,1,4.924833e+08,1.0,----------------------------------------------...,"[(3643, 3649)]","""Skin: No rashes or lesions. Neurological: Awa...","[""memory""]",1
2,100000272,10040002098,Z6352398,3.082899e+09,2015-10-15 00:00:00.0000000,Office Visit,640846456,Progress Notes,1,6.127209e+08,1.0,----------------------------------------------...,"[(6049, 6057)]","""She passed secondary to this. He does not kno...","[""cerebral""]",1
3,100000272,10040002098,Z6352398,3.085967e+09,2015-08-20 00:00:00.0000000,Office Visit,563095698,Progress Notes,1,5.395701e+08,1.0,----------------------------------------------...,"[(3810, 3816)]","""Skin: No rashes or lesions. Neurological: Awa...","[""memory""]",1
4,100000272,10040002098,Z6352398,3.094749e+09,2015-10-14 00:00:00.0000000,Abstract,638983522,Progress Notes,1,6.109901e+08,1.0,----------------------------------------------...,"[(5554, 5562)]","""She passed secondary to this. He does not kno...","[""cerebral""]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,100000894,200766,Z6352922,3.161210e+09,2017-06-22 00:00:00.0000000,Hospital Encounter,1658446121,Op Note,1,1.607201e+09,1.0,----------------------------------------------...,"[(2419, 2427)]","""nucleus was slowly and completely removed usi...","[""cortical""]",1
96,100000894,200766,Z6352922,3.172606e+09,2017-09-27 00:00:00.0000000,Hospital Encounter,1770603844,Op Note,1,1.722130e+09,1.0,----------------------------------------------...,"[(3575, 3583)]","""tunnel completely. It was checked carefully t...","[""cortical""]",1
97,100000894,200766,Z6352922,3.181499e+09,2018-03-28 00:00:00.0000000,Office Visit,1916263837,Progress Notes,1,1.871982e+09,1.0,----------------------------------------------...,"[(1209, 1215), (1248, 1256), (1292, 1296), (52...","""Rest of plan detailed in A&P section. PMH: Pa...","[""Memory"", ""dementia"", ""MOCA""]",3
98,100000894,200766,Z6352922,3.181499e+09,2018-03-28 00:00:00.0000000,Office Visit,1916263837,Progress Notes,1,1.871982e+09,1.0,----------------------------------------------...,"[(1209, 1215), (1248, 1256), (1292, 1296), (52...","""PMH: Patient Active Problem List Diagnosis Da...","[""Memory"", ""dementia"", ""MOCA""]",3


In [31]:
# to get # of general encouters --> do len(df[df["keyword_count"] >= 1])
# to get # of such encounters per patient
    # get list of unique patient ids
    # for each one, do len(df[df["patient_id"] == curr_patient_id]["encounter_id"])

In [67]:
encounters_with_more_than_1_keyword = notes[notes['match_count'] >= 1]
encounters_with_more_than_1_keyword

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count
0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,Office Visit,471433064,Progress Notes,1,4.532520e+08,1.0,----------------------------------------------...,"[(5850, 5858)]","""She passed secondary to this. He does not kno...","[""cerebral""]",1
1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,Office Visit,513142600,Progress Notes,1,4.924833e+08,1.0,----------------------------------------------...,"[(3643, 3649)]","""Skin: No rashes or lesions. Neurological: Awa...","[""memory""]",1
2,100000272,10040002098,Z6352398,3.082899e+09,2015-10-15 00:00:00.0000000,Office Visit,640846456,Progress Notes,1,6.127209e+08,1.0,----------------------------------------------...,"[(6049, 6057)]","""She passed secondary to this. He does not kno...","[""cerebral""]",1
3,100000272,10040002098,Z6352398,3.085967e+09,2015-08-20 00:00:00.0000000,Office Visit,563095698,Progress Notes,1,5.395701e+08,1.0,----------------------------------------------...,"[(3810, 3816)]","""Skin: No rashes or lesions. Neurological: Awa...","[""memory""]",1
4,100000272,10040002098,Z6352398,3.094749e+09,2015-10-14 00:00:00.0000000,Abstract,638983522,Progress Notes,1,6.109901e+08,1.0,----------------------------------------------...,"[(5554, 5562)]","""She passed secondary to this. He does not kno...","[""cerebral""]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270292,100365175,10059358563,Z8714148,3.360340e+09,2021-04-23 00:00:00.0000000,Telemedicine,5693313879,Progress Notes,1,5.690061e+09,1.0,----------------------------------------------...,"[(964, 970)]","""past 3 years. Interval events He was last see...","[""memory""]",1
270293,100365175,10059358563,Z8714148,3.363845e+09,2021-05-10 00:00:00.0000000,Office Visit,5784497372,Progress Notes,1,5.781962e+09,1.0,----------------------------------------------...,"[(391, 400)]","""and a moderate mixed hearing loss rising to w...","[""cognition""]",1
270294,100365175,10059358563,Z8714148,3.363845e+09,2021-05-10 00:00:00.0000000,Office Visit,5784981138,Progress Notes,1,5.782449e+09,1.0,----------------------------------------------...,"[(1194, 1203)]","""of Onset \u2022 Other Mother frequent falls \...","[""Alzheimer""]",1
270295,100365175,10059358563,Z8714148,3.366388e+09,2021-04-22 00:00:00.0000000,Follow-Up,5684835347,Progress Notes,1,5.681518e+09,1.0,----------------------------------------------...,"[(672, 678), (7858, 7867)]","""recent months. He even thinks that on May 29t...","[""memory""]",1
